In [13]:
import numpy as np
from sklearn.preprocessing import scale, robust_scale, minmax_scale
from scipy import stats
import scipy.stats
import pandas as pd
import statsmodels.api as sm #그래프 출력시
import matplotlib as mpl #그래프 옵션
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import graphviz
import os
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.formula.api import ols
from statsmodels.tools.eval_measures import rmse
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import scale
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from datetime import datetime
from sklearn import metrics

import matplotlib.pyplot as plt #그래프 출력시
import matplotlib as mpl #그래프 옵션

#그래프의 한글을 더욱 선명하게 출력
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

#그래프의 음수값이 나올때, 깨지는 현상 방지
mpl.rc('axes', unicode_minus=False)

#그래프 시각화 옵션
%matplotlib inline
#그래프 한글 설정
mpl.rc('font',family="Malgun Gothic")
plt.rcParams['axes.unicode_minus']=False

In [2]:
ds_tr = pd.read_excel('/home/pirl/Desktop/data.xlsx',sheetname='app_transaction')
ds_tr.info()
ds_us = pd.read_excel('/home/pirl/Desktop/data.xlsx',sheetname='app_user')
ds_us.info()

#csv로 읽을 시 엔진=파이썬 제거해야 칼럼 1로 안 읽음
#tr은 결측치 없이 모든 열이 10000개의 관측치 존재
#us는 결측치가 있는 열이 많음

/home/pirl/anaconda3/lib/python3.7/site-packages/pandas/io/excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
회원코드          10000 non-null object
프로그램          10000 non-null object
상태            10000 non-null object
프로그램가격        10000 non-null int64
최종결제금액        10000 non-null int64
결제시각          10000 non-null object
결제유형          10000 non-null object
채널            10000 non-null object
보조식품가격        10000 non-null int64
보조식품(추가구매)    10000 non-null object
기기종류          10000 non-null object
기기제조사         10000 non-null object
OSversion     10000 non-null object
dtypes: int64(3), object(10)
memory usage: 1015.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
회원코드              10000 non-null object
성별                9869 non-null float64
연령                9869 non-null float64
신장                9869 non-null float64
회원초기체중            9869 non-null float64
최저체중              5777 non-null float64
목표체중              9869 no

In [36]:
ds_us.isnull().sum()
#ds_us의 결측치 갯수

회원코드                 0
성별                 131
연령                 131
신장                 131
회원초기체중             131
최저체중              4223
목표체중               131
당뇨여부(설문)             6
음주여부(설문)             0
일주일운동횟수(설문)          0
건강보조식품복용여부(설문)       0
dtype: int64

In [4]:
print(ds_tr.columns)
print(ds_us.columns)

Index(['회원코드', '프로그램', '상태', '프로그램가격', '최종결제금액', '결제시각', '결제유형', '채널',
       '보조식품가격', '보조식품(추가구매)', '기기종류', '기기제조사', 'OSversion'],
      dtype='object')
Index(['회원코드', '성별', '연령', '신장', '회원초기체중', '최저체중', '목표체중', '당뇨여부(설문)',
       '음주여부(설문)', '일주일운동횟수(설문)', '건강보조식품복용여부(설문)'],
      dtype='object')


In [5]:
print(ds_tr.index)
print(ds_us.index)

RangeIndex(start=0, stop=10000, step=1)
RangeIndex(start=0, stop=10000, step=1)


In [11]:
#데이터프레임 상에서 칼럼 나오는 출력 갯수 늘리기
pd.options.display.max_columns=100

In [12]:
print(ds_tr.values)
print(ds_us.values)

#us의 미입력 -> 결측치 처리 Nan으로 바꿔야 함


[['Y9RY2VSI' '체중감량 프로그램 (타입B)' 'completed' ... 'iPhone' 'Apple'
  'iOS11.4.1']
 ['3GTN3S3B' '체중감량 프로그램 (타입B)' 'completed' ... 'SM-N935S' 'samsung'
  'Android7.0']
 ['6B0IG276' '체중감량 프로그램 (타입 A)' 'completed' ... 'iPhone' 'Apple'
  'iOS12.0']
 ...
 ['VWJ4NLZY' '체중감량 프로그램 (타입 A)' 'completed' ... 'iPhone' 'Apple'
  'iOS11.2.1']
 ['NJ2PR967' '체중감량 프로그램 (타입B)' 'cancelled' ... 'SM-G955N' 'samsung'
  'Android8.0.0']
 ['KVX64CPM' '체중감량 프로그램 (타입 A)' 'cancelled' ... 'SM-N960N' 'samsung'
  'Android8.1.0']]
[['Y9RY2VSI' 2.0 25.0 ... '미입력' '5회이상' '있음']
 ['3GTN3S3B' 1.0 26.0 ... '미입력' '5회이상' '있음']
 ['6B0IG276' 2.0 23.0 ... '마시지않음' '5회이상' '있음']
 ...
 ['VWJ4NLZY' 2.0 28.0 ... '미입력' '3~4회' '없음']
 ['NJ2PR967' 1.0 28.0 ... '미입력' '5회이상' '있음']
 ['KVX64CPM' 2.0 26.0 ... '미입력' '3~4회' '없음']]


In [22]:
ds_tr.head(2)

,회원코드,프로그램,상태,프로그램가격,최종결제금액,결제시각,결제유형,채널,보조식품가격,보조식품(추가구매),기기종류,기기제조사,OSversion
0,Y9RY2VSI,체중감량 프로그램 (타입B),completed,112500,112500,2017-04-14 오후 7:03:29.976000,Recurring,others,0,구매안함,iPhone,Apple,iOS11.4.1
1,3GTN3S3B,체중감량 프로그램 (타입B),completed,44780,44780,2017-05-23 오후 8:53:54.368000,Recurring,others,0,구매안함,SM-N935S,samsung,Android7.0


In [18]:
ds_tr.tail(2)

,회원코드,프로그램,상태,프로그램가격,최종결제금액,결제시각,결제유형,채널,보조식품가격,보조식품(추가구매),기기종류,기기제조사,OSversion
9998,NJ2PR967,체중감량 프로그램 (타입B),cancelled,112500,129500,2017-04-28 오전 7:05:19.776000,Recurring,facebook,0,구매안함,SM-G955N,samsung,Android8.0.0
9999,KVX64CPM,체중감량 프로그램 (타입 A),cancelled,112500,112500,2017-06-02 오전 10:16:34.576000,Recurring,others,0,구매안함,SM-N960N,samsung,Android8.1.0


In [21]:
ds_us.head(2)

,회원코드,성별,연령,신장,회원초기체중,최저체중,목표체중,당뇨여부(설문),음주여부(설문),일주일운동횟수(설문),건강보조식품복용여부(설문)
0,Y9RY2VSI,2.0,25.0,172.0,66.9,65.8,55.0,없음,미입력,5회이상,있음
1,3GTN3S3B,1.0,26.0,176.0,70.0,NaN,65.0,없음,미입력,5회이상,있음


In [20]:
ds_us.tail(2)

,회원코드,성별,연령,신장,회원초기체중,최저체중,목표체중,당뇨여부(설문),음주여부(설문),일주일운동횟수(설문),건강보조식품복용여부(설문)
9998,NJ2PR967,1.0,28.0,188.0,110.0,NaN,80.0,없음,미입력,5회이상,있음
9999,KVX64CPM,2.0,26.0,167.0,57.5,NaN,51.0,없음,미입력,3~4회,없음


In [26]:
ds_tr['결제유형'].head()

0    Recurring
1    Recurring
2    Recurring
3    Recurring
4    Recurring
Name: 결제유형, dtype: object

In [23]:
a=ds_tr[['결제유형','채널']]
print(a.shape)
a.head()

(10000, 2)


,결제유형,채널
0,Recurring,others
1,Recurring,others
2,Recurring,facebook
3,Recurring,naver
4,Recurring,facebook


In [35]:
ds_tr['결제유형'].value_counts()
#ds_tr['결제유형']가 모두 Recurring 으로 똑같은 값, 의미가 없음 따라서 삭제 필요


Recurring    10000
Name: 결제유형, dtype: int64

In [34]:
print("프로그램가격 최대: ", ds_tr['프로그램가격'].max())
print("최종결제금액 최대: ", ds_tr['최종결제금액'].max())
print("보조식품가격 최대: ", ds_tr['보조식품가격'].max())
print("프로그램가격 최소: ", ds_tr['프로그램가격'].min())
print("최종결제금액 최소: ", ds_tr['최종결제금액'].min())
print("보조식품가격 최소: ", ds_tr['보조식품가격'].min())
print("--------------------------------------")
print("성별 최대: ",ds_us['성별'].max())
print("연령 최대: ",ds_us['연령'].max())
print("신장 최대: ",ds_us['신장'].max())
print("회원초기체중 최대: ",ds_us['회원초기체중'].max())
print("최저체중 최대: ",ds_us['최저체중'].max())
print("목표체중 최대: ",ds_us['목표체중'].max())
print("성별 최소: ",ds_us['성별'].min())
print("연령 최소: ",ds_us['연령'].min())
print("신장 최소: ",ds_us['신장'].min())
print("회원초기체중 최소: ",ds_us['회원초기체중'].min())
print("최저체중 최소: ",ds_us['최저체중'].min())
print("목표체중 최소: ",ds_us['목표체중'].min())

#tr에는 max, min에 

프로그램가격 최대:  177000
최종결제금액 최대:  210000
보조식품가격 최대:  40000
프로그램가격 최소:  44780
최종결제금액 최소:  44780
보조식품가격 최소:  0
--------------------------------------
성별 최대:  2.0
연령 최대:  173.0
신장 최대:  203.2
회원초기체중 최대:  153.0
최저체중 최대:  152.6
목표체중 최대:  137.0
성별 최소:  1.0
연령 최소:  0.0
신장 최소:  -1.0
회원초기체중 최소:  30.0
최저체중 최소:  5.0
목표체중 최소:  -1.0


In [40]:
ds_us['성별'].value_counts()
#성별 입력 data에는 2, 1 외에는 없음

2.0    8846
1.0    1023
Name: 성별, dtype: int64

In [52]:
ds=pd.merge(ds_tr, ds_us, on='회원코드', how='outer')
ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 23 columns):
회원코드              10000 non-null object
프로그램              10000 non-null object
상태                10000 non-null object
프로그램가격            10000 non-null int64
최종결제금액            10000 non-null int64
결제시각              10000 non-null object
결제유형              10000 non-null object
채널                10000 non-null object
보조식품가격            10000 non-null int64
보조식품(추가구매)        10000 non-null object
기기종류              10000 non-null object
기기제조사             10000 non-null object
OSversion         10000 non-null object
성별                9869 non-null float64
연령                9869 non-null float64
신장                9869 non-null float64
회원초기체중            9869 non-null float64
최저체중              5777 non-null float64
목표체중              9869 non-null float64
당뇨여부(설문)          9994 non-null object
음주여부(설문)          10000 non-null object
일주일운동횟수(설문)       10000 non-null object
건강보조식품복용여부(설문)  

# ㅇ

In [53]:
ds.groupby('성별')['신장'].agg('mean')
#성별1이 남자인 것으로 추정, 주민등록번호 1과 2

성별
1.0    175.831965
2.0    161.729681
Name: 신장, dtype: float64

In [ ]:
ds['성별_Nan']=ds[['성별']=='Nan']

In [ ]:
#4개 다 결측치인 경우
ds_us[